<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/knu_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 필요 라이브러리 설치
!pip install PyPDF2 chromadb sentence-transformers pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.2 MB/s eta 0

In [ ]:
# 구글 드라이브 document_files에 있는 파일들 전부 저장
import os
import re
import unicodedata
import pytz
import pandas as pd
from datetime import datetime
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
from chromadb import PersistentClient

# 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
ROOT_FOLDER = "/content/drive/MyDrive/document_files"
URL_MAPPING_SUFFIX = "_url.xlsx"

KST = pytz.timezone('Asia/Seoul')
model = SentenceTransformer(EMBEDDING_MODEL_NAME)

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)
existing_ids = set(collection.get(limit=None)['ids'])

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[■◆●※▶▷▲→◇]', '', text)
    return text.strip()

def split_chunks(text, chunk_size=500):
    return [text[i:i+chunk_size].strip() for i in range(0, len(text), chunk_size)]

def read_file(filepath):
    if filepath.endswith('.pdf'):
        reader = PdfReader(filepath)
        return "".join([p.extract_text() for p in reader.pages if p.extract_text()])
    elif filepath.endswith('.md'):
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    return ""

def get_all_files(folder):
    all_files = []
    for dirpath, _, filenames in os.walk(folder):
        for f in filenames:
            if f.endswith(('.pdf', '.md')):
                all_files.append(os.path.join(dirpath, f))
    return all_files

def extract_category(file_path, base_folder):
    rel_path = os.path.relpath(file_path, start=base_folder)
    parts = rel_path.split(os.sep)
    return parts[0] if len(parts) > 1 else "기타"

def build_metadata(file_path, base_folder, base_name, source_url, ext):
    category = extract_category(file_path, base_folder)
    department_path = os.path.relpath(base_folder, ROOT_FOLDER)

    return {
        "file_name": base_name,
        "department": department_path,
        "category": category,
        "source_type": "pdf" if ext == ".pdf" else "markdown",
        "source_url": source_url,
        "date": datetime.now(KST).isoformat()
    }

total_added = 0

# 엑셀 기준으로 저장
for dirpath, _, filenames in os.walk(ROOT_FOLDER):
    for filename in filenames:
        if filename.endswith(URL_MAPPING_SUFFIX):
            folder = dirpath  # 엑셀 위치 폴더 = 기준 폴더
            dept_folder_name = os.path.basename(folder)
            url_file = os.path.join(folder, filename)

            try:
                url_df = pd.read_excel(url_file)
                url_mapping = dict(zip(url_df['파일명'], url_df['URL']))
            except Exception as e:
                print(f"❌ URL 매핑 로드 실패: {url_file} ({e})")
                continue

            file_paths = get_all_files(folder)

            for file_path in file_paths:
                file_name = os.path.basename(file_path)
                base_name = unicodedata.normalize("NFC", os.path.splitext(file_name)[0])
                ext = os.path.splitext(file_name)[1].lower()

                if file_name.endswith(URL_MAPPING_SUFFIX):
                    continue

                raw_text = read_file(file_path)
                if not raw_text.strip():
                    print(f"⚠️ {file_name}: 텍스트 없음 → 스킵")
                    continue

                cleaned = clean_text(raw_text)
                chunks = split_chunks(cleaned)
                embeddings = model.encode(chunks).tolist()
                ids = [f"{dept_folder_name}_{base_name}_chunk_{i}" for i in range(len(chunks))]

                new_chunks, new_embeddings, new_ids, new_metadatas = [], [], [], []

                for chunk, emb, id_ in zip(chunks, embeddings, ids):
                    if id_ not in existing_ids:
                        meta = build_metadata(file_path, folder, base_name, url_mapping.get(base_name, "출처 URL 미등록"), ext)
                        new_chunks.append(chunk)
                        new_embeddings.append(emb)
                        new_ids.append(id_)
                        new_metadatas.append(meta)

                if new_chunks:
                    collection.add(
                        documents=new_chunks,
                        embeddings=new_embeddings,
                        metadatas=new_metadatas,
                        ids=new_ids
                    )
                    print(f"✅ {file_name}: {len(new_chunks)}개 저장 완료")
                    total_added += len(new_chunks)
                else:
                    print(f"⚠️ {file_name}: 중복 청크만 존재 → 스킵")

print(f"\n🎉 총 저장된 신규 청크 수: {total_added}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

⚠️ 소프트웨어전공 졸업규정.pdf: 중복 청크만 존재 → 스킵
⚠️ SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문.pdf: 중복 청크만 존재 → 스킵

🎉 총 저장된 신규 청크 수: 0


In [ ]:
from chromadb import PersistentClient

# 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# 전체 문서 불러오기
results = collection.get(limit=None, include=["documents", "metadatas", "embeddings"])

print(f"\n📦 전체 저장된 벡터 수: {len(results['ids'])}")
print("=" * 70)

for i in range(len(results["ids"])):
    doc_id = results["ids"][i]
    metadata = results["metadatas"][i]
    doc_text = results["documents"][i].strip().replace("\n", " ")
    embedding = results["embeddings"][i]

    print(f"🔹 ID: {doc_id}")
    print(f"   📁 파일명: {metadata.get('file_name')}")
    print(f"   🏫 부서: {metadata.get('department')}")
    print(f"   📂 카테고리: {metadata.get('category')}")
    print(f"   📄 타입: {metadata.get('source_type')}")
    print(f"   🌐 URL: {metadata.get('source_url')}")
    print(f"   📅 날짜: {metadata.get('date')[:10]}")
    print(f"   💬 청크 문장: {doc_text}")
    print(f"   🧠 벡터 길이: {len(embedding)}")
    print(f"   🔢 벡터 앞 5개 값: {embedding[:5]}") # 384개 중에 앞 5개만 보여줌
    print("-" * 70)


📦 전체 저장된 벡터 수: 24
🔹 ID: Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_0
   📁 파일명: 소프트웨어전공 졸업규정
   🏫 부서: department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)
   📂 카테고리: 규정자료실
   📄 타입: pdf
   🌐 URL: https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426
   📅 날짜: 2025-04-01
   💬 청크 문장: - 7 -소프트웨어전공 졸업논문 규정 제1조(목적) 이 내규는 공주대학교 학칙(이하 “학칙”이라 함) 제94조 및 공주대학교 학사운영 규정(이하 “학사규정 ”이라 함) 제12장에 의거 소프트웨어전공 , 컴퓨터소프트웨어공학전공 및 멀티미디어공학전공의 졸업논문에 관한 세부사항을 규정함을 목적으로 한다. 제2조(졸업논문 제출자격 ) ① 최종학기 등록을 필한 자라야 한다. ② 3~4학년 재학중 공모전 출품을 1회 하여야 한다(출품인 4인까지 인정), 제3조(졸업논문 계획서 제출) ① 졸업논문 제출자격자는 4학년 1학기(최종학기 직전학기 ) 개시 후 2주내에 졸업논문 계획서[별지서식 (1)]를 논문지도교수의 지도를 받아 전공주임에게 제출 하여 승인을 받아야 한다. ② 지도교수는 졸업논문의 자료준비 , 작성 및 발표 등에 관한 사항을 지도한다 . 제4조(졸업논문 제출) ① 지도교수에게 2개 학기 이상 논문 지도를 받고 학술대회 제 1저자 발 표실적이 1건 이상 있어야하며 , 최종학기 수업 3/
   🧠 벡터 길이: 384
   🔢 벡터 앞 5개 값: [-0.02203827  0.06772523  0.01900545  0.01029711 -0.0526075 ]
---------

In [ ]:
# 완전 초기화 ------ 신중히 사용하기 바람 -------
"""
from chromadb import PersistentClient

PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)

# 컬렉션 삭제
client.delete_collection(name=COLLECTION_NAME)
print("🧨 컬렉션 전체 삭제 완료 (초기화됨)")

# 다시 생성
client.create_collection(name=COLLECTION_NAME)
print("✅ 새 빈 컬렉션 재생성 완료")
"""

'\nfrom chromadb import PersistentClient\n\nPERSIST_DIR = "/content/drive/MyDrive/chroma_index"\nCOLLECTION_NAME = "knu_collection"\n\nclient = PersistentClient(path=PERSIST_DIR)\n\n# 컬렉션 삭제\nclient.delete_collection(name=COLLECTION_NAME)\nprint("🧨 컬렉션 전체 삭제 완료 (초기화됨)")\n\n# 다시 생성\nclient.create_collection(name=COLLECTION_NAME)\nprint("✅ 새 빈 컬렉션 재생성 완료")\n'